In [43]:
from PIL import Image
import math
import sys
from matplotlib import pyplot as plt
import cv2
import scipy.stats

In [44]:
def splitpixels(img):
    pixRow = []
    pix = []
    pixNum = 0

    pixels = img.getdata()

    for pixel in pixels:
        if isinstance(pixel, int):
            pixRow.append(pixel)
        else:
            pixRow.append(pixel[0])

        pixNum += 1

        if pixNum % img.size[0] == 0:
            pix.append(pixRow)
            pixRow = []
    
    print(pixNum, "pixels")

    return pix



In [45]:
def groupmask(gmask, mask):

    # initialise the new mask variable
    newgroupmask = []

    # adds to the list for the new mask
    for line in gmask:
        newgroupmask.append(list(line))

    # sets the row and column values
    totalrow = len(newgroupmask)
    totalcolumn = len(newgroupmask[0])

    # adds/subtracts from the group depending on if it is divisible by 2 (mod2)
    for row in range(0, totalrow):
        for column in range(0, totalcolumn):
            if newgroupmask[row][column] % 2 == 0:
                newgroupmask[row][column] += mask[row][column]
            else:
                newgroupmask[row][column] -= mask[row][column]

    # returns the calculated mask
    return newgroupmask

In [46]:
def discrimination_function(group):

    # initialise variables
    amount = 0
    totalrow = len(group)
    totalcolumn = len(group[0])

    # the discrimination function is based on the work of friedrich et al
    # cycles through the columns using the discrimination function
    for row in range(0, totalrow):
        for column in range(0, totalcolumn):
            if column < (totalcolumn - 1):
                amount += abs(group[row][column] - group[row][column + 1])
    # cycles through the rows using the discrimination function
    for column in range(0, totalcolumn):
        for row in range(0, totalrow):
            if row < (totalrow - 1):
                amount += abs(group[row][column] - group[row + 1][column])
    return amount

In [47]:
def breakimage(imagearray, maskk, position):
    # initiate a new list
    brokeimage = []

    # adds each line to the list
    for line in maskk:
        brokeimage.append(list(line))

    # cycles through the image with the chosen mask to break the image
    for temprow in range(0, len(maskk)):
        for tempcol in range(0, len(maskk[0])):
            brokeimage[temprow][tempcol] = imagearray[temprow + position[0]][tempcol + position[1]]

    return brokeimage

In [48]:
def analyseLSBs(imageBox, mask, neg_mask, discriminator_overlap):

    r_p2 = 0
    s_p2 = 0
    r_1p2 = 0
    s_1p2 = 0
    neg_r_p2 = 0
    neg_s_p2 = 0
    neg_r_1p2 = 0
    neg_s_1p2 = 0
    
    imageRow = len(imageBox)
    imageCol = len(imageBox[0])
    
    maskRow = len(mask)
    maskCol = len(mask[0])

    if discriminator_overlap:
        num = float((imageRow - maskRow + 1) * (imageCol - maskCol + 1))
    else:
        num = float((imageRow - imageRow % maskRow) / maskRow * (imageCol - imageCol % maskCol) / maskCol)

    print("number of groups to check = ", int(num))

    numCount = 0

    for row in range(0, imageRow):
        for column in range(0, imageCol):
            if discriminator_overlap:
                if row <= imageRow - maskRow:
                    if column <= imageCol - maskCol:
                        pos = [row, column]
                        numCount += 1
                        breakimagebox = breakimage(imageBox, mask, pos)

                        flip_box = []
                        for line in breakimagebox:
                            flip_box.append(list(line))
                        for fliprow in range(0, len(breakimagebox)):
                            for flipcolumn in range(0, len(breakimagebox[0])):
                                if breakimagebox[fliprow][flipcolumn] % 2 == 0:
                                    flip_box[fliprow][flipcolumn] += 1
                                elif breakimagebox[fliprow][flipcolumn] % 2 == 1:
                                    flip_box[fliprow][flipcolumn] += -1
                        
                        discr_breakimagebox = discrimination_function(breakimagebox)
                        discr_mask_breakimagebox = discrimination_function(groupmask(breakimagebox, mask))
                        discr_neg_mask_breakimagebox = discrimination_function(groupmask(breakimagebox, neg_mask))
                        discr_flip_box = discrimination_function(flip_box)
                        discr_mask_flip_box = discrimination_function(groupmask(flip_box, mask))
                        discr_neg_mask_flip_box = discrimination_function(groupmask(flip_box, neg_mask))

                        

                        if discr_breakimagebox > discr_mask_breakimagebox:
                            s_p2 += 1
                        elif discr_breakimagebox < discr_mask_breakimagebox:
                            r_p2 += 1

                        if discr_breakimagebox > discr_neg_mask_breakimagebox:
                            neg_s_p2 += 1
                        elif discr_breakimagebox < discr_neg_mask_breakimagebox:
                            neg_r_p2 += 1

                        if discr_flip_box > discr_mask_flip_box:
                            s_1p2 += 1
                        elif discr_flip_box < discr_mask_flip_box:
                            r_1p2 += 1

                        if discr_flip_box < discr_neg_mask_flip_box:
                            neg_r_1p2 += 1
                        elif discr_flip_box > discr_neg_mask_flip_box:
                            neg_s_1p2 += 1

                        
                        if numCount % 1000 == 0:
                            sys.stdout.write('\rgroups checked so far = ' + str(numCount))

            else:
                if (row + 1) % maskRow == 0:
                    if (column + 1) % maskCol == 0:
                        # this is the start of the group
                        pos = [row - maskRow + 1, column - maskCol + 1]
                        numCount += 1
                        breakimagebox = breakimage(imageBox, mask, pos)

                        flip_box = []
                        for line in breakimagebox:
                            flip_box.append(list(line))
                        for fliprow in range(0, len(breakimagebox)):
                            for flipcolumn in range(0, len(breakimagebox[0])):
                                if breakimagebox[fliprow][flipcolumn] % 2 == 0:
                                    flip_box[fliprow][flipcolumn] += 1
                                elif breakimagebox[fliprow][flipcolumn] % 2 == 1:
                                    flip_box[fliprow][flipcolumn] += -1

                        discr_breakimagebox = discrimination_function(breakimagebox)
                        discr_mask_breakimagebox = discrimination_function(groupmask(breakimagebox, mask))
                        discr_neg_mask_breakimagebox = discrimination_function(groupmask(breakimagebox, neg_mask))
                        discr_flip_box = discrimination_function(flip_box)
                        discr_mask_flip_box = discrimination_function(groupmask(flip_box, mask))
                        discr_neg_mask_flip_box = discrimination_function(groupmask(flip_box, neg_mask))

                        if discr_breakimagebox > discr_mask_breakimagebox:
                            s_p2 += 1
                        elif discr_breakimagebox < discr_mask_breakimagebox:
                            r_p2 += 1

                        if discr_breakimagebox > discr_neg_mask_breakimagebox:
                            neg_s_p2 += 1
                        elif discr_breakimagebox < discr_neg_mask_breakimagebox:
                            neg_r_p2 += 1

                        if discr_flip_box > discr_mask_flip_box:
                            s_1p2 += 1
                        elif discr_flip_box < discr_mask_flip_box:
                            r_1p2 += 1

                        if discr_flip_box < discr_neg_mask_flip_box:
                            neg_r_1p2 += 1
                        elif discr_flip_box > discr_neg_mask_flip_box:
                            neg_s_1p2 += 1

                        if numCount % 1000 == 0:
                            sys.stdout.write('\rgroups checked so far = ' + str(numCount))

    print('\rgroups checked  = ', numCount)

    if num == 0:
        return 0

    d0 = float(r_p2 - s_p2) / num
    dn0 = float(neg_r_p2 - neg_s_p2) / num
    d1 = float(r_1p2 - s_1p2) / num
    dn1 = float(neg_r_1p2 - neg_s_1p2) / num

    a = 2 * (d1 + d0)
    b = (dn0 - dn1 - d1 - 3 * d0)
    c = (d0 - dn0)

    if b * b < 4 * a * c:
        # avoid negative root
        message_length = 0
    elif a == 0:
        # avoid deviding by zero
        message_length = 0
    else:
        # x = (-b+- sqrt(b^2-4ac))/2a, quadratic
        quadratic_solution1 = (-b + math.sqrt(b * b - 4 * a * c)) / (2 * a)
        quadratic_solution2 = (-b - math.sqrt(b * b - 4 * a * c)) / (2 * a)
        if abs(quadratic_solution1) < abs(quadratic_solution2):
            quadratic_solution = quadratic_solution1
        else:
            quadratic_solution = quadratic_solution2
        # p = x/(x−1/2), where p is message length
        message_length = abs(quadratic_solution / (quadratic_solution - 0.50))

    return message_length

In [49]:
def image_analyser(img, chosen_mask, discriminator_overlap):
    
    neg_mask = []

    for l in chosen_mask:
        neg_mask.append(list(l))

    for r in range(len(neg_mask)):
        for c in range(len(neg_mask[0])):
            if neg_mask[r][c] == 1:
                neg_mask[r][c] = -1
            elif neg_mask[r][c] == -1:
                neg_mask[r][c] = 1

    pix = splitpixels(img)

    # displays the size of the chosen mask
    print("")
    print("Mask size = ", len(chosen_mask[0]), "x", len(chosen_mask))

    # analyses the red pixels to determine what percent of them may contain embedded content
    print("")
    print("Analysing Red LSBs")
    gpercent = analyseLSBs(pix, chosen_mask, neg_mask, discriminator_overlap)

    # controls any errors within the calculations
    print("")
    if gpercent == 0:
        print("Unable to calculate the percent of the pixels")

        print("")
        encodedpercent = "?"
    else:
        # calculates and displays the total percentage of pixels that are likely to be encoded
        encodedpercent = gpercent
        print("")
        print("Decimal value probability of pixels likely to be encoded: ", encodedpercent)
        totalpercent = (encodedpercent * 100)
        print("Total Percent of pixels likely to contain embedded data: ", round(totalpercent, 2), "%")
        width, height = img.size
        totalpix = int(width) * int(height)
        # the size of the file is calculated by multiplying the percent of encoded pixels by the total number of pixels
        # then this is multiplied by 3 as each pixel requires 3 bits (rgb)
        data = ((encodedpercent * totalpix) * 3)
        print("Approximately ", round(data, 2), " bits of data (", round((data/8000), 2), "KB)")

    return encodedpercent

In [50]:
m0 = [[0, 1, 0]]
discrimination_overlap = 0

In [51]:
filename = "./pup.jpg"
image_filename = Image.open(filename)
image_analyser(image_filename, m0, discrimination_overlap)

99600 pixels

Mask size =  3 x 1

Analysing Red LSBs
number of groups to check =  33000
groups checked  =  3300033000


Decimal value probability of pixels likely to be encoded:  0.007463587463947514
Total Percent of pixels likely to contain embedded data:  0.75 %
Approximately  2230.12  bits of data ( 0.28 KB)


0.007463587463947514

In [53]:
filename = "./pvd6_img.png"
image_filename = Image.open(filename)
image_analyser(image_filename, m0, discrimination_overlap)

99600 pixels

Mask size =  3 x 1

Analysing Red LSBs
number of groups to check =  33000
groups checked  =  3300033000


Decimal value probability of pixels likely to be encoded:  0.024641263930817923
Total Percent of pixels likely to contain embedded data:  2.46 %
Approximately  7362.81  bits of data ( 0.92 KB)


0.024641263930817923